In [1]:
import pandas as pd
import sqlite3 as sq
import duckdb
from utils.quack import Quack

In [5]:
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',]
cols2roll = ["completions", "attempts", "carries", "passing_yards", "rushing_yards", "receiving_yards"]

In [6]:
weekly = Quack.fetch_table('weekly')
per_game = weekly.groupby(['game_id','season','week','team', 'opponent_team'])[stat_cols].sum().reset_index().sort_values(by=['season','week'])
per_game['pass_pct'] = per_game['attempts'] / (per_game['attempts'] + per_game['carries'])
per_game['yards_per_carry'] = per_game['rushing_yards'] / per_game['carries']
per_game['yards_per_reception'] = per_game['receiving_yards'] / per_game['completions']
cols2roll += ['yards_per_carry', 'yards_per_reception', 'pass_pct']

In [7]:
roll_stats = [x + '_team_roll' for x in cols2roll]
print(roll_stats)
team = per_game.copy()
team[roll_stats] = team.groupby("team")[
    cols2roll
].transform(lambda x: x.rolling(7, min_periods=1).mean()).sort_values(by='completions')
team[roll_stats] = team[roll_stats] / team[roll_stats].mean()
team[['team','season','week'] + roll_stats]

['completions_team_roll', 'attempts_team_roll', 'carries_team_roll', 'passing_yards_team_roll', 'rushing_yards_team_roll', 'receiving_yards_team_roll', 'yards_per_carry_team_roll', 'yards_per_reception_team_roll', 'pass_pct_team_roll']


,team,season,week,completions_team_roll,attempts_team_roll,carries_team_roll,passing_yards_team_roll,rushing_yards_team_roll,receiving_yards_team_roll,yards_per_carry_team_roll,yards_per_reception_team_roll,pass_pct_team_roll
0,ARI,2000,1,1.354520,1.466021,0.743171,1.353418,0.379033,1.353437,0.518590,0.986994,1.284730
1,NYG,2000,1,0.822387,0.747970,1.523501,0.732037,1.965683,0.732048,1.311918,0.879275,0.685269
2,BAL,2000,1,0.870763,0.957401,1.337708,0.846950,1.234061,0.846962,0.938018,0.960784,0.851346
3,PIT,2000,1,0.822387,1.166833,0.668854,0.846950,0.264442,0.846962,0.402008,1.017301,1.237812
4,CAR,2000,1,0.822387,0.777889,0.743171,0.778854,0.987249,0.778865,1.350746,0.935508,1.022540
...,...,...,...,...,...,...,...,...,...,...,...,...
12953,SEA,2025,3,0.981336,0.867645,0.891805,0.941191,0.849991,0.941204,0.998069,0.952075,0.985696
12954,NYJ,2025,3,0.981336,0.944579,0.897114,0.919303,1.021249,0.919316,1.158288,0.957180,1.022613
12955,TB,2025,3,1.071176,0.957401,1.093523,1.023272,1.269320,1.023286,1.182012,0.951168,0.936462
12956,NE,2025,3,1.057355,0.936031,0.955506,0.975239,0.901620,0.975253,0.966324,0.912991,0.982588


In [9]:
opp_stats = [x + '_opp_roll' for x in cols2roll]
opp = per_game.copy()
opp[opp_stats] = opp.groupby("opponent_team")[
    cols2roll
].transform(lambda x: x.rolling(7, min_periods=1).mean()).sort_values(by='completions')
opp[opp_stats] = opp[opp_stats] / opp[opp_stats].mean()
opp[['opponent_team','season','week'] + opp_stats]

,opponent_team,season,week,completions_opp_roll,attempts_opp_roll,carries_opp_roll,passing_yards_opp_roll,rushing_yards_opp_roll,receiving_yards_opp_roll,yards_per_carry_opp_roll,yards_per_reception_opp_roll,pass_pct_opp_roll
0,NYG,2000,1,1.355154,1.466781,0.743769,1.354069,0.379277,1.354064,0.518360,0.987198,1.283976
1,ARI,2000,1,0.822772,0.748358,1.524727,0.732390,1.966948,0.732387,1.311336,0.879457,0.684867
2,PIT,2000,1,0.871170,0.957898,1.338785,0.847358,1.234855,0.847354,0.937602,0.960983,0.850846
3,BAL,2000,1,0.822772,1.167438,0.669392,0.847358,0.264612,0.847354,0.401829,1.017512,1.237085
4,WAS,2000,1,0.822772,0.778292,0.743769,0.779228,0.987884,0.779225,1.350146,0.935702,1.021940
...,...,...,...,...,...,...,...,...,...,...,...,...
12953,NO,2025,3,1.009451,0.927964,1.216594,0.961109,1.222255,0.961106,1.020172,0.961430,0.877837
12954,TB,2025,3,1.085506,1.034872,0.823459,1.037146,0.589706,1.037142,0.743716,0.989664,1.108289
12955,NYJ,2025,3,0.988709,0.940793,1.136905,0.936169,1.132791,0.936165,1.002889,0.950440,0.912187
12956,PIT,2025,3,1.126990,0.940793,1.200656,1.070603,1.353301,1.070599,1.076748,0.991718,0.895780


In [10]:
team[['game_id','team','season','week'] + roll_stats].to_csv('data/agg/team_stats.csv')
opp[['game_id','opponent_team','season','week'] + opp_stats].to_csv('data/agg/opp_stats.csv')

In [11]:

con = duckdb.connect("data/nfl.duckdb")

team = team[['game_id','team','season','week'] + roll_stats]
team.to_csv('data/agg/team_stats.csv')
opp = opp[['game_id','opponent_team','season','week'] + opp_stats]
opp.to_csv('data/agg/opp_stats.csv')

con.execute("CREATE OR REPLACE TABLE team_feats AS SELECT * FROM team")
con.execute("CREATE OR REPLACE TABLE opp_feats AS SELECT * FROM opp")

con.commit()
con.close()

In [12]:
team.columns

Index(['game_id', 'team', 'season', 'week', 'completions_team_roll',
       'attempts_team_roll', 'carries_team_roll', 'passing_yards_team_roll',
       'rushing_yards_team_roll', 'receiving_yards_team_roll',
       'yards_per_carry_team_roll', 'yards_per_reception_team_roll',
       'pass_pct_team_roll'],
      dtype='object')

In [13]:
opp

,game_id,opponent_team,season,week,completions_opp_roll,attempts_opp_roll,carries_opp_roll,passing_yards_opp_roll,rushing_yards_opp_roll,receiving_yards_opp_roll,yards_per_carry_opp_roll,yards_per_reception_opp_roll,pass_pct_opp_roll
0,2000_01_ARI_NYG,NYG,2000,1,1.355154,1.466781,0.743769,1.354069,0.379277,1.354064,0.518360,0.987198,1.283976
1,2000_01_ARI_NYG,ARI,2000,1,0.822772,0.748358,1.524727,0.732390,1.966948,0.732387,1.311336,0.879457,0.684867
2,2000_01_BAL_PIT,PIT,2000,1,0.871170,0.957898,1.338785,0.847358,1.234855,0.847354,0.937602,0.960983,0.850846
3,2000_01_BAL_PIT,BAL,2000,1,0.822772,1.167438,0.669392,0.847358,0.264612,0.847354,0.401829,1.017512,1.237085
4,2000_01_CAR_WAS,WAS,2000,1,0.822772,0.778292,0.743769,0.779228,0.987884,0.779225,1.350146,0.935702,1.021940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12953,2025_03_NO_SEA,NO,2025,3,1.009451,0.927964,1.216594,0.961109,1.222255,0.961106,1.020172,0.961430,0.877837
12954,2025_03_NYJ_TB,TB,2025,3,1.085506,1.034872,0.823459,1.037146,0.589706,1.037142,0.743716,0.989664,1.108289
12955,2025_03_NYJ_TB,NYJ,2025,3,0.988709,0.940793,1.136905,0.936169,1.132791,0.936165,1.002889,0.950440,0.912187
12956,2025_03_PIT_NE,PIT,2025,3,1.126990,0.940793,1.200656,1.070603,1.353301,1.070599,1.076748,0.991718,0.895780


In [15]:
team

,game_id,team,season,week,completions_team_roll,attempts_team_roll,carries_team_roll,passing_yards_team_roll,rushing_yards_team_roll,receiving_yards_team_roll,yards_per_carry_team_roll,yards_per_reception_team_roll,pass_pct_team_roll
0,2000_01_ARI_NYG,ARI,2000,1,1.354520,1.466021,0.743171,1.353418,0.379033,1.353437,0.518590,0.986994,1.284730
1,2000_01_ARI_NYG,NYG,2000,1,0.822387,0.747970,1.523501,0.732037,1.965683,0.732048,1.311918,0.879275,0.685269
2,2000_01_BAL_PIT,BAL,2000,1,0.870763,0.957401,1.337708,0.846950,1.234061,0.846962,0.938018,0.960784,0.851346
3,2000_01_BAL_PIT,PIT,2000,1,0.822387,1.166833,0.668854,0.846950,0.264442,0.846962,0.402008,1.017301,1.237812
4,2000_01_CAR_WAS,CAR,2000,1,0.822387,0.777889,0.743171,0.778854,0.987249,0.778865,1.350746,0.935508,1.022540
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12953,2025_03_NO_SEA,SEA,2025,3,0.981336,0.867645,0.891805,0.941191,0.849991,0.941204,0.998069,0.952075,0.985696
12954,2025_03_NYJ_TB,NYJ,2025,3,0.981336,0.944579,0.897114,0.919303,1.021249,0.919316,1.158288,0.957180,1.022613
12955,2025_03_NYJ_TB,TB,2025,3,1.071176,0.957401,1.093523,1.023272,1.269320,1.023286,1.182012,0.951168,0.936462
12956,2025_03_PIT_NE,NE,2025,3,1.057355,0.936031,0.955506,0.975239,0.901620,0.975253,0.966324,0.912991,0.982588


In [14]:
roll_stats

['completions_team_roll',
 'attempts_team_roll',
 'carries_team_roll',
 'passing_yards_team_roll',
 'rushing_yards_team_roll',
 'receiving_yards_team_roll',
 'yards_per_carry_team_roll',
 'yards_per_reception_team_roll',
 'pass_pct_team_roll']